# 04 - PDE Finite-Difference Pricing

Price European options with finite differences (Crank-Nicolson) and compare with Black-Scholes.

In [10]:
%cd /content
!rm -rf interactive_portfolio_optimization
!git clone https://github.com/basarr/interactive_portfolio_optimization.git
!ls /content/interactive_portfolio_optimization


/content
Cloning into 'interactive_portfolio_optimization'...
remote: Enumerating objects: 71, done.
remote: Counting objects: 100% (71/71), done.
remote: Compressing objects: 100% (65/65), done.
remote: Total 71 (delta 24), reused 42 (delta 4), pack-reused 0 (from 0)
Receiving objects: 100% (71/71), 46.36 KiB | 3.31 MiB/s, done.
Resolving deltas: 100% (24/24), done.
data  notebooks  pyproject.toml  README.md  results  src  tests


In [11]:
from pathlib import Path
print((Path("/content/interactive_portfolio_optimization/src")).exists())
print((Path("/content/interactive_portfolio_optimization/pyproject.toml")).exists())


True
True


In [12]:
from pathlib import Path
import sys

ROOT = Path("/content/interactive_portfolio_optimization")
if str(ROOT) not in sys.path:
    sys.path.insert(0, str(ROOT))
print("ROOT OK:", ROOT)


ROOT OK: /content/interactive_portfolio_optimization


In [13]:
from pathlib import Path
import sys

# Auto-find project root by locating pyproject.toml + src
ROOT = None
for pp in Path("/content").rglob("pyproject.toml"):
    cand = pp.parent
    if (cand / "src").exists():
        ROOT = cand
        break

if ROOT is None:
    raise FileNotFoundError("Could not find project root with /src under /content. Clone repo first.")

if str(ROOT) not in sys.path:
    sys.path.insert(0, str(ROOT))

for p in [ROOT/"results", ROOT/"results"/"tables", ROOT/"results"/"figures", ROOT/"results"/"logs", ROOT/"results"/"reports"]:
    p.mkdir(parents=True, exist_ok=True)

print("ROOT:", ROOT)
print("src exists:", (ROOT / "src").exists())


ROOT: /content/interactive_portfolio_optimization
src exists: True


In [15]:
import pandas as pd

from src.config import config_dict
from src.pde_fd import fd_convergence_vs_bs
from src.plotting import plot_pde_error

cfg = config_dict(fast_mode=True)
print("imports + cfg ready")


imports + cfg ready


In [16]:
grid_pairs = [(50, 50), (100, 100), (200, 200), (300, 300)]
pde_df = fd_convergence_vs_bs(
    S0=cfg['S0'], K=cfg['K'], r=cfg['R'], q=cfg['Q'], sigma=cfg['SIGMA'], T=cfg['T'],
    option_type='call', S_max=cfg['S0'] * cfg['S_MAX_MULT'], grid_pairs=grid_pairs, scheme='CN'
)
pde_df.to_csv(ROOT / 'results' / 'tables' / 'pde_convergence.csv', index=False)
plot_pde_error(pde_df, ROOT / 'results' / 'figures' / 'pde_error_vs_grid.png')
pde_df


,M,N,fd_price,bs_price,abs_error
0,50,50,8.986449,8.916037,0.070411
1,100,100,8.932989,8.916037,0.016952
2,200,200,8.920346,8.916037,0.004309
3,300,300,8.913595,8.916037,0.002442


Notebook 04 Summary (practical + simple)

We priced the same European call option using two approaches:
	1.	Finite-difference solution of the pricing PDE (Crank–Nicolson scheme).
	2.	Black–Scholes closed-form solution (benchmark).

The key PDE idea is to solve the Black–Scholes equation backward in time:

$\frac{\partial V}{\partial t} + \frac{1}{2}\sigma^2 S^2 \frac{\partial^2 V}{\partial S^2} + (r-q)S\frac{\partial V}{\partial S} - rV = 0,$
for $t \in [0,T]$, $S \in (0,\infty)$.

with terminal condition:

$V(S,T) = \max(S-K,0)$ (call),

$V(S,T)=\max(K-S,0)$ (put).

Backward time-marching:

Choose a grid $S_j = j\Delta S$ and $t_n = n\Delta t$ with $\Delta t = T/N$.

Initialize at maturity: $V_j^{N} = \max(S_j - K,0)$.
Then for $n = N-1, N-2, \dots, 0$, compute $V^{n}$ from $V^{n+1}$ by solving the finite-difference system (Crank–Nicolson form):

$(I - \tfrac{\Delta t}{2}L),V^{n} = (I + \tfrac{\Delta t}{2}L),V^{n+1} + \text{BC terms},$

where $V^{n} = (V_0^{n},\dots,V_M^{n})^\top$ and $L$ is the discrete spatial operator approximating
$L V = \tfrac{1}{2}\sigma^2 S^2 V_{SS} + (r-q)S V_S - rV.$

That equation is the PDE analogue of backward induction: you start from $t=T$ and step to $t=0$.

Using the grid recursion $(I - \tfrac{\Delta t}{2}L)V^{n} = (I + \tfrac{\Delta t}{2}L)V^{n+1} + \text{BC terms}$, we start from the terminal payoff vector $V^{N}$ and solve a tridiagonal linear system at each step to obtain $V^{N-1}, V^{N-2}, \dots, V^{0}$. The final PDE price is the value at $t=0$ interpolated at $S=S_0$, i.e. $V(S_0,0)$.


Findings from the numerical grid:

•	$(M = N = 50)$: error $\approx 0.0704$

•	$(M = N = 100)$: error $\approx 0.0170$

•	$(M = N = 200)$: error $\approx 0.0043$

•	$(M = N = 300)$: error $\approx 0.0024$



# *Interpretation (why this matches the scheme)*

•	As $M$ (spatial nodes) increases, the finite-difference approximations to $V_S$ and $V_{SS}$ become more accurate, so the discrete operator $L$ better matches the continuous PDE operator.

•	As $N$ (time steps) increases, the time-marching error decreases, and Crank–Nicolson’s averaging between levels $n$ and $n+1$ improves accuracy for a fixed spatial grid.

•	The monotone drop in $|V_{\text{PDE}}-V_{\text{BS}}|$ is consistent with numerical convergence of the discretization toward the analytic benchmark.

Practical takeaway
•	Coarse grids (small $M,N$) are fast but can produce noticeable numerical error.

•	Moderate/fine grids give prices close to the benchmark, with the remaining gap dominated by discretization plus boundary truncation effects from solving on $S \in [0,S_{\max}]$ instead of $(0,\infty)$.

•	With $(M,N)=(300,300)$ the error is already small ($\approx 0.0024$), suggesting the implementation is stable and correctly wired (terminal condition, boundary conditions, and backward time-marching).